In [1]:
import json
import os
from pathlib import Path
from matplotlib import pyplot as plt
from PIL import Image
import random
import shutil
import numpy as np
import glob
import h5py
from scipy import ndimage
from pathlib import Path

In [2]:
class Args():
    resize = (224,224)
args = Args()

In [8]:
datafiles = '/home/yuchen/Swin-MAE/land4sensor/ValidData'
output_image_path = '/home/yuchen/Swin-MAE/land4sensor/resplit_data/'

files = glob.glob(datafiles + '/img/*')

mean_ = [-0.4914, -0.3074, -0.1277, -0.0625, 0.0439, 0.0803, 0.0644, 0.0802, 0.3000, 0.4082, 0.0823, 0.0516, 0.3338, 0.7819]
std_ = [0.9325, 0.8775, 0.8860, 0.8869, 0.8857, 0.8418, 0.8354, 0.8491, 0.9061, 1.6072, 0.8848, 0.9232, 0.9018, 1.2913]

files = glob.glob(datafiles + '/img/*')

img_lst = []
label_lst = []
bbox_lst = []
final_img_lst = []
id_idx = []
mask_lst = []

num_image = len(files)

for idx in range(num_image):
    
    id = files[idx].split('_')[1].split('.')[0]
    id_idx.append(id)
    mask_file_name = f'{datafiles}/mask/mask_{id}.h5'
    
    with h5py.File(files[idx], 'r') as hf:
        image = hf['img'][:]
        image = np.asarray(image, np.float32)
        image = image.transpose((-1, 0, 1))
        for i in range(len(mean_)):
            image[i,:,:] -= mean_[i]
            image[i,:,:] /= std_[i]
        img_lst.append(image)
        
img_lst = np.array(img_lst)

for channel_idx in range(img_lst.shape[1]):
    for img_idx in range(img_lst.shape[0]):
        channel = img_lst[img_idx,channel_idx]
        min_val = np.nanmin(channel)
        max_val = np.nanmax(channel)
        if max_val!=min_val:
            img_lst[img_idx, channel_idx] = (img_lst[img_idx, channel_idx] - min_val) / (max_val - min_val)

for i in range(num_image):
    img = img_lst[i][np.array([3,2,1])]
    original_h, original_w = img.shape[1], img.shape[2]
    resize_h, resize_w = args.resize
    
    image = Image.fromarray(np.transpose(np.array((img.transpose((2,1,0))*255), dtype=np.uint8),(1,0,2)), 'RGB')
    
    if args.resize != None:
        image = image.resize(args.resize)

    filename = str(id_idx[i])
    image.save(output_image_path+str(filename)+'.jpg', 'JPEG')


In [9]:
num_image

245